<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: <br>
   **Student ID**: <br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [51]:
# libraries
import csv
import pandas as pd
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

### loading dataset
This dataset is Tesla's stock from 2010 to 2020. it has some columns of opening price or highest and lowest price of the day. we try to predict closing/the last price of the stock based on the other features.


In [83]:
df = pd.read_csv('TSLA.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900
...,...,...,...,...,...,...,...
2411,2020-01-28,568.489990,576.809998,558.080017,566.900024,566.900024,11788500
2412,2020-01-29,575.690002,589.799988,567.429993,580.989990,580.989990,17801500
2413,2020-01-30,632.419983,650.880005,618.000000,640.809998,640.809998,29005700
2414,2020-01-31,640.000000,653.000000,632.520020,650.570007,650.570007,15719300


In [84]:
df = df.drop(['Date'],axis=1)
y = df["Close"]
X = df.drop(["Close","Adj Close"],axis=1)

### Split dataset to train and test

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Regression mdoel closed form

In [5]:
class RegressionCloseForm:
    def __init__(self):
        self.W = None
              
    def train(self, X_train, y):
        X = np.append(np.ones((X_train.shape[0], 1)), X_train , axis=1)
        self.W = np.dot((np.linalg.inv(np.dot(X.T,X))), np.dot(X.T,y))
    
    def predict(self, X_test):
        X = np.append(np.ones((X_test.shape[0], 1)), X_test , axis=1)
        return np.dot(X, self.W)
    
    def mse_loss(self, pred, real):
        diff = pred - real
        return np.sum(diff * diff) / diff.size

In [6]:
model = RegressionCloseForm()
model.train(X_train, y_train)
predictions = model.predict(X_test)
model.mse_loss(y_test, predictions)

4.37943489437885

### Vanilla Linear regression with sklearn

In [7]:
regr_model = linear_model.LinearRegression()
regr_model.fit(X_train, y_train)

LinearRegression()

MSE error on training data

In [8]:
predictions_train = regr_model.predict(X_train)
mean_squared_error(y_train, predictions_train)

4.057920329276168

MSE error on test data

In [9]:
predictions = regr_model.predict(X_test)
mean_squared_error(y_test, predictions)

4.379434894551934

In [10]:
# Evaluation of model regression score function. it shows our model works well
regr_model.score(X_test, y_test)

0.9997117604197343

Compare predicted values with real ones

In [11]:
comparison = pd.DataFrame({'Predicted':predictions,'Real':y_test, 'differnece':np.abs(predictions - y_test)})
comparison

,Predicted,Real,differnece
410,31.442062,31.490000,0.047938
199,24.529889,24.650000,0.120111
1670,278.905101,279.760010,0.854909
1934,330.673522,328.200012,2.473510
1036,262.081422,259.320007,2.761415
...,...,...,...
1423,174.974500,179.000000,4.025500
1458,254.608833,251.860001,2.748832
281,24.919015,25.059999,0.140984
1804,341.245531,345.660004,4.414473


In [145]:
class RegressionModel:
    
    def __init__(self, lr, number_of_epochs, regularization=None):
        self.m = None # samples
        self.n = None # features
        self.W = None # weight
        self.regularization = regularization # penalty object
        self.lr = lr # learning rate
        self.epoch = number_of_epochs # iteration
        
    def __calculate_cost(self, y, y_pred):
        diff = y_pred - y
        ss = np.square(diff)
        cost = (1 / (2 * self.m)) * np.sum(ss)
        if self.regularization:
            return cost + self.regularization(self.W) 
        return cost
    
    def __initialize(self, X):
        X2 = np.insert(X.to_numpy().astype(np.float64), 0, 1, axis=1)
        self.m, self.n = X2.shape
        #self.W = np.zeros((self.n, 1))
        self.W = np.random.rand(self.n, 1)
        return X2
    
    def __update_weights(self, X, y, y_pred):
        dw = (1 / self.m) * np.dot(X.T, (y_pred - y))
        if self.regularization:
            dw += self.regularization.derivation(self.W)
        self.W = self.W - self.lr * dw
        
    def fit(self, X_train, y):
        X = self.__initialize(X_train)
        y = y.to_numpy().astype(np.float64)
        for e in tqdm.tqdm(range(1, self.epoch+1)):
            y_pred = np.dot(X, self.W)
            cost = self.__calculate_cost(y, y_pred)
            self.__update_weights(X, y, y_pred)
            if e % 1000 == 0:
                print(f"The Cost in iteration {e}----->{cost} :)")

    def predict(self, X_test):
        X_test2 = np.insert(X_test.to_numpy(), 0 , 1, axis= 1)
        return np.dot(X_test2, self.W) 

In [146]:
y_pred.shape

(1932, 1)

In [147]:
y_train

1666    269.200012
1563    196.050003
544      30.730000
1025    223.570007
821     180.949997
           ...    
1638    214.679993
1095    242.589996
1130    220.970001
1294    255.250000
860     120.500000
Name: Close, Length: 1932, dtype: float64

In [148]:
xx = X_train.copy()
y2 = y_train.to_numpy()
xx = np.insert(xx.to_numpy(), 0, 1, axis=1)
w = np.random.rand(xx.shape[1], 1)
y_pred = np.dot(xx, w)
cost = (1 / (2 * xx.shape[0])) * np.sum(np.square(y_pred - y2))

In [149]:
cost

4.2966408832374776e+16

In [150]:
class ElasticPenalty:
    
    def __init__(self, l = 0.1, l_ratio = 0.5):
        self.l = l 
        self.l_ratio = l_ratio

    def __call__(self, w):
        l1_contribution = self.l_ratio * self.l * np.sum(np.abs(w))
        l2_contribution = (1 - self.l_ratio) * self.l * 0.5 * np.sum(np.square(w))
        return (l1_contribution + l2_contribution)

    def derivation(self, w):
        l1_derivation = self.l * self.l_ratio * np.sign(w)
        l2_derivation = self.l * (1 - self.l_ratio) * w
        return (l1_derivation + l2_derivation)

In [151]:
class LassoRegression(RegressionModel):
        def __init__(self, lr, number_of_epochs, l):
            regularization = ElasticPenalty(l, l_ratio=1) # penalty object
            super().__init__(lr, number_of_epochs, regularization)

            
class RidgeRegression(RegressionModel):
        def __init__(self, lr, number_of_epochs, l):
            regularization = ElasticPenalty(l, l_ratio=0) # penalty object
            super().__init__(lr, number_of_epochs, regularization)

            
class ElasticNet(RegressionModel):
        def __init__(self, lr, number_of_epochs, l = 0.1, l_ratio = 0.5):
            regularization = ElasticPenalty(l, l_ratio) # penalty object
            super().__init__(lr, number_of_epochs, regularization)

In [152]:
def mse_loss(pred, real):
    diff = pred - real
    return np.sum(diff * diff) / diff.size

In [153]:
reg_model_custom = RegressionModel(lr=1e-2, number_of_epochs=100)
reg_model_custom.fit(X_train, y_train)
predictions = reg_model_custom.predict(X_test)
mse_loss(y_test, predictions)

 12%|█████                                     | 12/100 [00:00<00:02, 40.73it/s]/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/2242044821.py:13: RuntimeWarning: overflow encountered in square
  ss = np.square(diff)
 22%|█████████▏                                | 22/100 [00:00<00:01, 41.41it/s]/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/2242044821.py:30: RuntimeWarning: invalid value encountered in subtract
  self.W = self.W - self.lr * dw
100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 41.57it/s]


ValueError: operands could not be broadcast together with shapes (484,) (484,1932) 

In [98]:
predictions

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [76]:
lasso_reg = LassoRegression(lr=1e-2, number_of_epochs=100, l=0.1)
lasso_reg.fit(X_train, y_train)
predictions = lasso_reg.predict(X_test)
mse_loss(y_test, predictions)

 13%|█████▍                                    | 13/100 [00:06<00:48,  1.81it/s]/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/2877816700.py:13: RuntimeWarning: overflow encountered in square
  cost = (1 / (2 * self.m)) * np.sum(np.square(y_pred - y))
 14%|█████▉                                    | 14/100 [00:06<00:46,  1.86it/s]/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/1362360954.py:9: RuntimeWarning: overflow encountered in square
  l2_contribution = (1 - self.l_ratio) * self.l * 0.5 * np.sum(np.square(w))
/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/1362360954.py:9: RuntimeWarning: invalid value encountered in double_scalars
  l2_contribution = (1 - self.l_ratio) * self.l * 0.5 * np.sum(np.square(w))
 27%|███████████▎                              | 27/100 [00:13<00:40,  1.82it/s]/var/folders/6p/5cn1t8t123n9_x4lpmlwgs1h0000gn/T/ipykernel_1931/1362360954.py:14: RuntimeWarning: invalid value encountered in multiply
  l2_derivat

ValueError: Shape of passed values is (484, 5), indices imply (484, 4)

array([[1.00000000e+00, 2.66250000e+02, 2.71179993e+02, 2.66149994e+02,
        7.82020000e+06],
       [1.00000000e+00, 1.97059998e+02, 1.98490005e+02, 1.93449997e+02,
        3.58940000e+06],
       [1.00000000e+00, 3.00000000e+01, 3.08500000e+01, 2.96500000e+01,
        1.47100000e+06],
       ...,
       [1.00000000e+00, 2.23809998e+02, 2.24320007e+02, 2.19520004e+02,
        4.50570000e+06],
       [1.00000000e+00, 2.60329987e+02, 2.60649994e+02, 2.55020004e+02,
        3.60430000e+06],
       [1.00000000e+00, 1.19379997e+02, 1.22720001e+02, 1.16099998e+02,
        1.38855000e+07]])

In [36]:
X_train.to_numpy()

array([[2.66250000e+02, 2.71179993e+02, 2.66149994e+02, 7.82020000e+06],
       [1.97059998e+02, 1.98490005e+02, 1.93449997e+02, 3.58940000e+06],
       [3.00000000e+01, 3.08500000e+01, 2.96500000e+01, 1.47100000e+06],
       ...,
       [2.23809998e+02, 2.24320007e+02, 2.19520004e+02, 4.50570000e+06],
       [2.60329987e+02, 2.60649994e+02, 2.55020004e+02, 3.60430000e+06],
       [1.19379997e+02, 1.22720001e+02, 1.16099998e+02, 1.38855000e+07]])

In [33]:
X_train.drop_duplicates(inplace=True)